In [130]:
!pip install pyperclip


  Running setup.py bdist_wheel for pyperclip ... done
  Stored in directory: /home/nikolay/.cache/pip/wheels/92/f0/ac/2ba2972034e98971c3654ece337ac61e546bdeb34ca960dc8c
Successfully built pyperclip
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [131]:
import random
import time
import uuid
import pyperclip


def generateName(length=5):
    return str(uuid.uuid4().hex)[:length]

class Service:
    def __init__(self, serviceId, idKey, propertyName, setup, name=None):
        self.serviceId = serviceId
        self.idKey = idKey
        self.serviceKey = idKey[:-3]
        self.propertyName = propertyName
        self.setup = setup
        self.name = generateName(4) if name == None else name

        
ids = {}

ids["ANALOG_ID"] = 1000
analog = Service(ids["ANALOG_ID"], "ANALOG_ID", "analog", "INPUT")

ids["ONOFF_ID"] = 1001
onoff = Service(ids["ONOFF_ID"], "ONOFF_ID", "onoff", "OUTPUT")

ids["TEMPERATURE_DHT11_ID"] = 1002
temperature11 = Service(ids["TEMPERATURE_DHT11_ID"], "TEMPERATURE_DHT11_ID", "temperature11", "INPUT")

ids["HUMIDITY_DHT11_ID"] = 1003
humidity11 = Service(ids["HUMIDITY_DHT11_ID"], "HUMIDITY_DHT11_ID", "humidity11", "INPUT")

ids["DIGITAL_ALERT_ID"] = 1004
alert = Service(ids["DIGITAL_ALERT_ID"], "DIGITAL_ALERT_ID", "alert", "INPUT")

ids["INIT_SERVICE_ID"] = 1005
init = Service(ids["INIT_SERVICE_ID"], "INIT_SERVICE_ID", "init", "INPUT")

ids["TEMPERATURE_DHT22_ID"] = 1006
temperature22 = Service(ids["TEMPERATURE_DHT22_ID"], "TEMPERATURE_DHT22_ID", "temperature22", "INPUT")

ids["HUMIDITY_DHT22_ID"] = 1007
humidity22 = Service(ids["HUMIDITY_DHT22_ID"], "HUMIDITY_DHT22_ID", "humidity22", "INPUT")

ids["PRESSURE_ID"] = 1008
pressure = Service(ids["PRESSURE_ID"], "PRESSURE_ID", "pressure", "INPUT")


In [132]:
deviceNameLength = 32
serviceNameLength = 8

In [133]:
deviceName = generateName(deviceNameLength)
rx, tx = "2", "3"
setup = [
    (analog, "A0"),
    (alert, "4"),
    (alert, "5"),
    (temperature22, "8"),
    (humidity22, "8"),
    (init, "9"),
    (onoff, "10"),
    (onoff, "11"),
    (onoff, "12")
]

In [135]:
paste = """
#include "user_properties.h"
#include "services.h"

#define DEVICE_NAME "%s"
#define RASPBERRY_PORT 8080
#define UDP_PORT       59743
#define ARDUINO_PORT   8080

// used services
""" % (deviceName)

for i in set(map(lambda x: x[0], setup)):
    paste += "#define " + i.serviceKey + "\n"


    
paste += "\n// pins for communication with esp8266\n"
paste += "#define RX " + rx + " // attach to TX of esp\n"
paste += "#define TX " + tx + " // attach to RX of esp\n"

paste += "\n// static services array, pay attention to the defined pins\n"
paste += "int SERVICES[] = {%s};\n" % ", ".join(map(lambda x: x[0].idKey, setup))
paste += "int PINS[] = {%s};\n" % ", ".join(map(lambda x: x[1], setup))
paste += "#define SERVICES_STR \"%s\" \n" % ";".join(map(lambda x: str(x[0].serviceId), setup)) 
paste += "#define SERVICE_NAMES_STR \"%s\" \n" % ";".join(map(lambda x: generateName(serviceNameLength), setup)) 
paste += "#define SERVICES_COUNT %s\n" % len(setup) 
paste += "#define DIGITAL_ALERT_COUNT %s\n" % list(map(lambda x: x[0], setup)).count(alert)
if (init in list(map(lambda x: x[0], setup))):
    paste += "#define INIT_PIN %s \n" % setup[list(map(lambda x: x[0], setup)).index(init)][1]
paste += "\n// will be called on arduino 'setup()'\n"
paste += "void setupConfiguration() {\n"
paste += "\tdelay(%i);\n" % random.randint(100, 5000)

for i in setup:
    paste += "\tpinMode(%s, %s);\n" % (i[1], i[0].setup)
paste += "}\n"

pyperclip.copy(paste)
print(paste)


#include "user_properties.h"
#include "services.h"

#define DEVICE_NAME "80a0f0a7946f4d3b8132bc7868be76dc"
#define RASPBERRY_PORT 8080
#define UDP_PORT       59743
#define ARDUINO_PORT   8080

// used services
#define INIT_SERVICE
#define ONOFF
#define ANALOG
#define DIGITAL_ALERT
#define TEMPERATURE_DHT22
#define HUMIDITY_DHT22

// pins for communication with esp8266
#define RX 2 // attach to TX of esp
#define TX 3 // attach to RX of esp

// static services array, pay attention to the defined pins
int SERVICES[] = {ANALOG_ID, DIGITAL_ALERT_ID, DIGITAL_ALERT_ID, TEMPERATURE_DHT22_ID, HUMIDITY_DHT22_ID, INIT_SERVICE_ID, ONOFF_ID, ONOFF_ID, ONOFF_ID};
int PINS[] = {A0, 4, 5, 8, 8, 9, 10, 11, 12};
#define SERVICES_STR "1000;1004;1004;1006;1007;1005;1001;1001;1001" 
#define SERVICE_NAMES_STR "29dd48d1;d7931d3f;180bc45e;210810cd;777c1ec0;5a96026d;2a1ac73d;f57be844;7a5357bf" 
#define SERVICES_COUNT 9
#define DIGITAL_ALERT_COUNT 2
#define INIT_PIN 9 

// will be called on arduino 'setup()'
v